# Code Executor - development notebook

## Docker container space

### Build and run

In [52]:
# Build image
!docker build . --quiet --tag code-executor:latest

sha256:e984519e169c81f26f83fc8b77390ed0cde0f9114cde05832db41a570476a330


In [55]:
# Run container
!docker run -d -p 8080:8080 --name test-code-executor code-executor:latest --host 0.0.0.0

070d7e55027ab723344677116c7e250f346dae697057f4020a8fbdead111acf6


In [3]:
# check container logs
!docker logs test-code-executor

Error response from daemon: No such container: test-code-executor


In [11]:
# kill and remove container
!docker kill test-code-executor && docker rm test-code-executor

test-code-executor
test-code-executor


In [6]:
# remove container
!docker rm test-code-executor

Error response from daemon: You cannot remove a running container 5f99f754e027c20b7a53f707579040df2419fbe51ab571172bff0ceaf79223f3. Stop the container before attempting removal or force remove


In [56]:
# list contexts to get docker host DNS
!docker context ls

NAME                             DESCRIPTION                               DOCKER ENDPOINT                                      ERROR
default                          Current DOCKER_HOST based configuration   unix:///var/run/docker.sock                          
m5.large_i-0543dc65b37b03cb1 *                                             tcp://ip-10-3-1-39.us-west-2.compute.internal:1111   


### Test container

In [63]:
# IP from docker context when using Studio Docker CLI extension
host_ip = "ip-10-3-1-39.us-west-2.compute.internal"

In [66]:
# Test ping
%%bash -s $host_ip
echo ${1}
curl ${1}:8080/ping

SyntaxError: invalid syntax (944402066.py, line 3)

In [67]:
%%bash -s $host_ip
curl http://${1}:8080/ping

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    22  100    22    0     0   2052      0 --:--:-- --:--:-- --:--:--  2000


{"Health_Check":"200"}

In [70]:
%%bash -s $host_ip
curl http://${1}:8080/execute_code -X POST -d '{"code": "print(\"Hello ok)"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   301  100   272  100    29    315     33 --:--:-- --:--:-- --:--:--   349


{"output":["Cell In[1], line 1\n    print(\"Hello ok)\n          ^\nSyntaxError: unterminated string literal (detected at line 1)",true],"error":["Cell In[1], line 1\n    print(\"Hello ok)\n          ^\nSyntaxError: unterminated string literal (detected at line 1)",true]}

In [59]:
# Test broken code execution
%%bash -s $host_ip
curl ${1}:8080/execute_code -X POST -d '{"code": "print(\"Hello ok)"}'

SyntaxError: invalid syntax (3883501092.py, line 3)

In [60]:
# test working code execution
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "print(\"Hello ok\")", "timeout": 10}'

SyntaxError: invalid syntax (3587098348.py, line 3)

In [8]:
# test variable persistance
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "a=10", "timeout": 10}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    58  100    27  100    31   3287   3774 --:--:-- --:--:-- --:--:--  7250


{"output":"","error":false}

In [9]:
# test variable persistance
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "print(a)", "timeout": 10}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    64  100    29  100    35   3200   3862 --:--:-- --:--:-- --:--:--  8000


{"output":"10","error":false}

In [10]:
# test restart kernel
%%bash -s $host_ip
curl -X POST ${1}:8080/restart_kernel -d '{"code": "print(\"Hello ok\")", "timeout": 10, "kernel_name": "ir"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   114  100    47  100    67    375    534 --:--:-- --:--:-- --:--:--   912


{"output":"Kernel ir restarted!","error":false}

In [11]:
# test listing kernel specs
%%bash -s $host_ip
curl ${1}:8080/list_kernel_specs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   349  100   349    0     0  37934      0 --:--:-- --:--:-- --:--:-- 43625


{"python3":{"display_name":"Python 3 (ipykernel)","language":"python"},"ir":{"display_name":"R","language":"R"},"julia-1.9":{"display_name":"Julia 1.9.3","language":"julia"},"javascript":{"display_name":"JavaScript (Node.js)","language":"javascript"},"bash":{"display_name":"Bash","language":"bash"},"java":{"display_name":"Java","language":"java"}}

In [41]:
# test java code
java_code = '\npublic class Main {\n  public static void main(String[] args) {\n    System.out.println(\"Hello World!\");  \n  }\n}\n'

In [42]:
import requests
import json
data = {"code": java_code, "timeout": 10, "kernel_name": "java"}
print(data)
requests.post(f"http://{host_ip}:8080/execute_code", data=json.dumps(data)).text

{'code': '\npublic class Main {\n  public static void main(String[] args) {\n    System.out.println("Hello World!");  \n  }\n}\n', 'timeout': 10, 'kernel_name': 'java'}


'{"output":"","error":false}'

In [43]:
import requests
import json
data = {"code": "Main.main(new String[] {\"\"})", "timeout": 10, "kernel_name": "java"}
print(data)
requests.post(f"http://{host_ip}:8080/execute_code", data=json.dumps(data)).text

{'code': 'Main.main(new String[] {""})', 'timeout': 10, 'kernel_name': 'java'}


'{"output":"Hello World!","error":false}'

In [21]:
%%bash -s $host_ip $java_code
echo ${2}
data='{"code": "'${2}'", "timeout": 10, "kernel_name": "java"}'
echo $data
curl -X POST ${1}:8080/execute_code -d ${data}

public
{"code": "public", "timeout": 10, "kernel_name": "java"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    29  100    21  100     8   5312   2023 --:--:-- --:--:-- --:--:--  9666
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: "public",
curl: (3) URL using bad/illegal format or missing URL
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: 10,
curl: (3) URL using bad/illegal format or missing URL
curl: (3) unmatched close brace/bracket in URL position 7:
"java"}
      ^


Internal Server Error

CalledProcessError: Command 'b'echo ${2}\ndata=\'{"code": "\'${2}\'", "timeout": 10, "kernel_name": "java"}\'\necho $data\ncurl -X POST ${1}:8080/execute_code -d ${data}\n'' returned non-zero exit status 3.

In [219]:
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "echo 1", "timeout": 10, "kernel_name": "bash"}'

{"output":"1","error":false}

In [12]:
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "(int) eval(\"1+2\")+3", "timeout": 10, "kernel_name": "java"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    99  100    28  100    71     26     66  0:00:01  0:00:01 --:--:--    93


{"output":"6","error":false}

In [246]:
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "console.log(\"Hello World\");", "timeout": 10, "kernel_name": "javascript"}'


{"output":"Hello World","error":false}

In [19]:
%%bash -s $host_ip
curl ${1}:8080/list_kernel_specs

curl: (6) Could not resolve host: ip-172-31-65-185.ap-southeast-2.compute.internal


In [45]:
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "numbers <- c(1, 4, 7, 3, 2)\navg <- mean(numbers)\nprint(avg)", "timeout": 10, "kernel_name": "ir"}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   143  100    34  100   109    393   1260 --:--:-- --:--:-- --:--:--  1662


{"output":"[1] 3.4","error":false}

In [40]:
!A=$"numbers <- c(1, 4, 7, 3, 2)\\navg <- mean(numbers)\nprint(avg)"; echo $A

numbers <- c(1, 4, 7, 3, 2)\navg <- mean(numbers)\nprint(avg)


In [ ]:
%%bash -s $host_ip
curl -X POST ${1}:8080/execute_code -d '{"code": "print(\"Hello World!\")", "timeout": 10, "kernel_name": "ir"}'

In [3]:
from jupyter_client import run_kernel

In [4]:
!jupyter kernelspec list

Available kernels:
  python3    /opt/conda/share/jupyter/kernels/python3


In [34]:
a = run_kernel(kernel_name="python3")

In [29]:
a.__dict__

{'gen': <generator object run_kernel at 0x7f567824b1b0>,
 'func': <function jupyter_client.manager.run_kernel(**kwargs: Any) -> Iterator[jupyter_client.client.KernelClient]>,
 'args': (),
 'kwds': {'kernel_name': 'python3'},
 '__doc__': 'Context manager to create a kernel in a subprocess.\n\n    The kernel is shut down when the context exits.\n\n    Returns\n    -------\n    kernel_client: connected KernelClient instance\n    '}

In [35]:
a1 = next(a.gen)
print(a1.execute("print('hello')"))
while True:
    try:
        msg = a1.get_iopub_msg(timeout=20)

        msg_type = msg["header"]["msg_type"]
        content = msg["content"]

        if msg_type == "execute_result":
            outputs.append(content["data"])
        elif msg_type == "stream":
            outputs.append(content["text"])
        elif msg_type == "error":
            error_flag = True
            outputs.append(content["traceback"])

        # If the execution state of the kernel is idle, it means the cell finished executing
        if msg_type == "status" and content["execution_state"] == "idle":
            break
    except:
        break
print(msg)

3ecbd2d1-1c918faad191aa2ba8d110aa_1031_2
{'header': {'msg_id': '028c91b8-c7e3980990078315a2ee15b2_1822_9', 'msg_type': 'stream', 'username': 'username', 'session': '028c91b8-c7e3980990078315a2ee15b2', 'date': datetime.datetime(2023, 11, 16, 4, 15, 36, 317538, tzinfo=tzlocal()), 'version': '5.3'}, 'msg_id': '028c91b8-c7e3980990078315a2ee15b2_1822_9', 'msg_type': 'stream', 'parent_header': {'msg_id': '3ecbd2d1-1c918faad191aa2ba8d110aa_1031_2', 'msg_type': 'execute_request', 'username': 'username', 'session': '3ecbd2d1-1c918faad191aa2ba8d110aa', 'date': datetime.datetime(2023, 11, 16, 4, 15, 36, 306969, tzinfo=tzlocal()), 'version': '5.3'}, 'metadata': {}, 'content': {'name': 'stdout', 'text': 'hello\n'}, 'buffers': []}


In [31]:
a

In [ ]:
class JupyterCell:
    def clean_output(self, outputs):
        outputs_only_str = list()
        for i in outputs:
            if type(i) == dict:
                if "text/plain" in list(i.keys()):
                    outputs_only_str.append(i["text/plain"])
            elif type(i) == str:
                outputs_only_str.append(i)
            elif type(i) == list:
                error_msg = "\n".join(i)
                error_msg = re.sub(r"\x1b\[.*?m", "", error_msg)
                outputs_only_str.append(error_msg)

        return "\n".join(outputs_only_str).strip()
    def run_cell

In [71]:
class JupyterNotebook:
    def __init__(self, kernel_name: str = "python3", startup_timeout: float = 60):
        self.kc = run_kernel(kernel_name=kernel_name).gen

    def clean_output(self, outputs):
        outputs_only_str = list()
        for i in outputs:
            if type(i) == dict:
                if "text/plain" in list(i.keys()):
                    outputs_only_str.append(i["text/plain"])
            elif type(i) == str:
                outputs_only_str.append(i)
            elif type(i) == list:
                error_msg = "\n".join(i)
                error_msg = re.sub(r"\x1b\[.*?m", "", error_msg)
                outputs_only_str.append(error_msg)

        return "\n".join(outputs_only_str).strip()

    def run_cell(self, code_string, timeout=10):
        # Execute the code and get the execution count
        outputs = []
        error_flag = False
        client = next(self.kc)
        msg_id = client.execute(code_string)

        while True:
            try:
                msg = client.get_iopub_msg(timeout=timeout)

                msg_type = msg["header"]["msg_type"]
                content = msg["content"]

                if msg_type == "execute_result":
                    outputs.append(content["data"])
                elif msg_type == "stream":
                    outputs.append(content["text"])
                elif msg_type == "error":
                    error_flag = True
                    outputs.append(content["traceback"])

                # If the execution state of the kernel is idle, it means the cell finished executing
                if msg_type == "status" and content["execution_state"] == "idle":
                    break
            except:
                print("here")
                break

        return self.clean_output(outputs), error_flag
    
s = JupyterNotebook()
s.run_cell("print('Hello World!')", 20) 

('Hello World!', False)